<h1><center>Sepsis Prediction</h1>
<h4>TCSS 555<br>
Spring 2018<br>
Thuan Lam, Tood Robbins, Inno Irving Estrera</h4></center>


<h2>Libraries</h2>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from dateutil.parser import parse
from datetime import datetime

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## Data

In [2]:
import os 
cwd = os.getcwd()
print('Current folder is {}'.format(cwd))

if os.name == "posix":
    unreadable = set([       
        "NEW_CHARTEVENTS.csv",
        "ADMISSIONS.csv",
        "NEW_ICUSTAYS.csv",
        "DRGCODES.csv"
    ])
    USER_DIR = '/Users/innoestrera/Desktop/mimic3/';
else:
    USER_DIR = 'Data\\';
print(USER_DIR)

Current folder is C:\Thuan\ML\Projects\Sepsis
Data\


In [3]:
# Load datasets
admissions = pd.read_csv("{}ADMISSIONS.csv".format(USER_DIR))[['HADM_ID','SUBJECT_ID']]
# convert Diagnosis to binary: 1 means sepsis
# admissions['DIAGNOSIS'] = admissions['DIAGNOSIS'].apply(lambda x : 0 if str(x).find('SEPSIS') == -1 else 1)
# print(admissions.head(2))
# adminssions = shuffle(stays)
patients = pd.read_csv("{}PATIENTS.csv".format(USER_DIR))[['SUBJECT_ID','GENDER','DOB']]
drugs = pd.read_csv("{}DRGCODES.csv".format(USER_DIR))[['HADM_ID','SUBJECT_ID','DRG_CODE','DESCRIPTION']]

stays = pd.read_csv("{}NEW_ICUSTAYS.csv".format(USER_DIR))[['HADM_ID','ICUSTAY_ID','LOS']]
# print(stays.head(1))

chart = pd.read_csv("{}NEW_CHARTEVENTS.csv".format(USER_DIR))
# print(chart.head(2))

## Create the Y Set

In [4]:
#get a copy with 2 columns only
find_sepsis = drugs[['HADM_ID', 'DRG_CODE']].copy()

#change drug code 870, 871, 872 to 1; Otherwise, 0
#https://www.icd10monitor.com/understanding-sepsis-an-example-of-the-convergence-of-clinical-quality-coding-reimbursement-and-audit
find_sepsis['DRG_CODE'] = find_sepsis['DRG_CODE'].apply(lambda x: 1 if (x >= 870 and x <= 872) or (x >= 867 and x <= 869) or x == 776 or (x >= 974 and x <= 976) else 0)

#sum all drugcodes grouup by HADM_ID. If the sum > 0 means HADM_ID has/had sepsis
find_sepsis = find_sepsis.groupby(['HADM_ID']).sum().reset_index() # .sort_values(by=['DRG_CODE'], ascending=False)

#convert DRG_CODE to binary: 1 means sepsis, 0 means NO
find_sepsis['DRG_CODE'] = find_sepsis['DRG_CODE'].apply(lambda x: 1 if x > 0 else 0)

#change DRG_CODR to SEPSIS, it would be easier 
find_sepsis.rename(columns={'DRG_CODE': 'SEPSIS'}, inplace=True)

# print('find_sepsis: ')
# print(find_sepsis.head(2))
# print('admission1: ')
# print(admissions.head(2))

#merge tables to create the Y set
admissions = pd.merge(admissions, find_sepsis, on='HADM_ID', how='left') #.drop(['SUBJECT_ID'], axis=1)
admissions = admissions.fillna(0)
# print('admission2: ')
# print(admissions.head(2))

# admissions.loc[admissions['SEPSIS'] > 0]

## Create the X Set

* <h3>Master

In [5]:
master = pd.merge(admissions, stays, on='HADM_ID', how='inner') #.drop(['SUBJECT_ID'], axis=1)
# print(master.head(3))

* <h3>Get ItemIDs from ChartEvents then Add into Master

In [ ]:
# get itemsIds from ChartEvent
for x in chart.ITEMID.unique():
    master[x] = 0

# set index
master = master.set_index('ICUSTAY_ID')
# print(master.head(2))

* <h3>Put Item's Values into Master

In [ ]:
for index, row in chart.iterrows():    
    master.loc[row['ICUSTAY_ID'], row['ITEMID']] = row['VALUE']
# print(master.head(2))

* <h3>Add Paitien Info.

In [ ]:
# process Patients table
patients['AGE'] = patients['DOB'].apply(lambda x: x[:4])
maxYear = int(patients['AGE'].max())

patients['AGE'] = patients['AGE'].apply(lambda x: maxYear - int(x))
patients['GENDER'] = patients['GENDER'].apply(lambda x: 0 if x == 'F' else 1)
patients = patients.drop(['DOB'], axis=1)

# add Patients into Master
master = pd.merge(master, patients, on='SUBJECT_ID', how='inner').drop(['SUBJECT_ID'], axis=1)

* <h3>Analyze Gender and Age

In [ ]:
sepsis_only = master[master['SEPSIS']==1]
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15,15))
fig.tight_layout() # Or equivalently,  "plt.tight_layout()"
fig.set_size_inches(20, 10)
plt.subplots_adjust(wspace=1, hspace=1)

# Gender
g = sepsis_only.groupby(['GENDER']).size().reset_index(name='GENDER_SEPSIS_COUNTS')
g['GENDER_COUNTS'] = master.groupby(['GENDER']).size().reset_index(name='GENDER_COUNTS')['GENDER_COUNTS']
g['GENDER_PERCENT'] = g.apply(lambda row: row.GENDER_SEPSIS_COUNTS / row.GENDER_COUNTS * 100, axis=1)
g = g.drop(columns=['GENDER_SEPSIS_COUNTS','GENDER_COUNTS'])
g.iloc[0, g.columns.get_loc('GENDER')] = 'Female'
g.iloc[1, g.columns.get_loc('GENDER')] = 'Male'
g = g.set_index('GENDER')
g.plot.bar(subplots=True, legend=None, ax=axes[0,0], rot=0, fontsize=12)

In [ ]:
# age
b1 = sepsis_only.groupby(['AGE']).size().reset_index(name='AGE_SEPSIS_COUNTS')
b2 = master.groupby(['AGE']).size().reset_index(name='AGE_COUNTS')
# show all records from master
b = pd.merge(b1, b2, on='AGE', how='right').fillna(0)
b['AGE_PERCENT'] = b.apply(lambda row: row.AGE_SEPSIS_COUNTS / row.AGE_COUNTS * 100, axis=1)
b = b.drop(columns=['AGE_SEPSIS_COUNTS','AGE_COUNTS'])
b = b.set_index('AGE')
b.plot.bar(legend=None, rot=0, figsize=(15,10), fontsize=12)


* <h3>Convert Age to Age_Group

In [ ]:
b = b.reset_index()
arr = []
for i in range(1000):
    arr.append(0)

for index, row in b.iterrows():
    age = int(row['AGE'])
    
    percent = row['AGE_PERCENT'] 
    print('age: {}  {}%'.format(age, percent))
    if percent < 3:
        arr[age] = 0
    elif percent < 6:
        arr[age] = 1
    elif percent < 9:
        arr[age] = 2
    elif percent < 12:
        arr[age] = 3
    elif percent < 15:
        arr[age] = 4
    else:    
        arr[age] = 5
        
master['AGE'] = master['AGE'].apply(lambda x : arr[x]) 

* <h3>Finalize the Master

In [ ]:
# move the SEPSIS column to the last 
cols = master.columns.tolist()
cols.insert(len(master.columns) - 1, cols.pop(cols.index('SEPSIS')))
master = master.reindex(columns= cols).reset_index()

# remove unnecessary columns
master.drop(['HADM_ID'], axis=1, inplace=True)
# print(master.head(2))
print('Master built. Shape: {}'.format(master.shape))
print(master.columns.values)

master.fillna(0, inplace=True)
master = shuffle(master)
# master = master[:30000]

# checking null
# ol_mask=master.isnull().any(axis=0) 
# ol_mask

if 'index' in master.columns:
    master.drop(['index'], axis=1, inplace=True)

## Baseline

In [ ]:
# sepsis_only = master[master['SEPSIS']==1]

# fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15,15))
# fig.tight_layout() # Or equivalently,  "plt.tight_layout()"
# fig.set_size_inches(20, 10)
# plt.subplots_adjust(wspace=1, hspace=1)

# # Gender
# g = sepsis_only.groupby(['SEPSIS']).size().reset_index(name='sepsis_counts')
# g['total_stays'] = master.groupby(['SEPSIS']).size().reset_index(name='total_stays')['total_stays']
# g['sepsis_percent'] = g.apply(lambda row: row.sepsis_counts / row.total_stays * 100, axis=1)
# g
# # g = g.drop(columns=['sepsis_counts','total_stays'])
# # g.iloc[0, g.columns.get_loc('SEPSIS')] = 'No Sepsis'
# # g.iloc[1, g.columns.get_loc('SEPSIS')] = 'Sepsis'
# # g = g.set_index('gender')
# # g.plot.bar(subplots=True, legend=None, ax=axes[0,0], rot=0, fontsize=12)

all_rows = master.shape[0]
sepsis_rows = len(master[(master['SEPSIS'] == 1)])
baseline = 1 -(sepsis_rows / all_rows)
print('Sepsis: {} rows out of {} rows'.format(sepsis_rows, all_rows))
print('Baseline: {}'.format(baseline))

In [ ]:
# master

## Model

In [ ]:
# Split-out validation dataset
col = len(master.columns) - 1
array = master.values   #numpy array
X = array[:,0:col]# first N columns
Y = array[:,col]  # SEPSIS column

In [ ]:
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)
# print('{}'.format(X_train, Y_train))

In [ ]:
# Test options and evaluation metric
seed = 7
scoring = 'accuracy'

# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('RF', RandomForestClassifier()))

# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Compare Algorithms
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
# Make predictions on validation dataset

messages = []
matrices = []
for name, model in models:
    model.fit(X_train, Y_train)
    prediction = model.predict(X_validation)
    
    accuracyScore = accuracy_score(Y_validation, prediction)
    confusionMatrix = confusion_matrix(Y_validation, prediction)
    matrices.append(confusionMatrix)

print(messages)
print(matrices)

## Conclusion

<h4> bla bla bla. <h4>

In [ ]:
# Make predictions on validation dataset
knn = KNeighborsClassifier()
knn.fit(X_train, Y_train)
predictions = knn.predict(X_validation)
# lr = LogisticRegression()
# lr.fit(X_train, Y_train)
# predictions = lr.predict(X_validation)
print(predictions)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

In [ ]:



# Make predictions on validation dataset
rf = RandomForestClassifier()
rf.fit(X_train, Y_train)
predictions = rf.predict(X_validation)
# lr = LogisticRegression()
# lr.fit(X_train, Y_train)
# predictions = lr.predict(X_validation)
print(predictions)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

In [ ]:
# Make predictions on validation dataset
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc
messages = []
matrices = []
for name, model in models:
    model.fit(X_train, Y_train)
    prediction = model.predict(X_validation)
    
    accuracyScore = accuracy_score(Y_validation, prediction)
    confusionMatrix = confusion_matrix(Y_validation, prediction)
    classificationReport = classification_report(Y_validation, prediction)
    precision, recall, thresholds = precision_recall_curve(Y_validation, prediction)
    sensitivity = confusionMatrix[0, 0]/(confusionMatrix[0, 0]+confusionMatrix[0, 1])
    specificity = confusionMatrix[1, 1]/(confusionMatrix[1, 0]+confusionMatrix[1, 1])
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_validation, prediction)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    #metrics.roc_curve(Y_validation, prediction)
    #auc = metrics.auc(confusionMatrix[0, 0], confusionMatrix[1, 1])
    
    print("accuracy", accuracyScore)
    print("confusion Matrix", confusionMatrix)
    print("precision", precision)
    print("recall", recall)
    print("sensitivity", sensitivity)
    print("specificity", specificity)
    print("stuff", roc_curve(Y_validation, prediction))
    plt.title('AUC')
    plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0,1],[0,1],'r--')
    plt.xlim([-0.1,1.2])
    plt.ylim([-0.1,1.2])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    #matrices.append(confusionMatrix)
    
#print(confusionMatrix)
#print(messages)